In [1]:
import os
pasta_campi = 'campi/'
lista_campi = []
lista_priori = []
for diretorios, subpastas, arquivos in os.walk(pasta_campi):
    lista_campi = [arquivo for arquivo in arquivos]

pasta_priori = 'priori/'
for diretorios, subpastas, arquivos in os.walk(pasta_priori):
    lista_priori = [arquivo for arquivo in arquivos]
    
# print(lista_campi)
# print(lista_priori)

dicionario = {
    'CAFI.xlsx': 'AFI - Afogados.xlsx',
    'CBAR.xlsx': 'BAR - Barreiros.xlsx',
    'CBJA.xlsx': 'BEJ - Belo Jardim.xlsx',
    'CGAR.xlsx': 'GAR - Garanhuns.xlsx',
    'CPES.xlsx': 'PES - Pesqueira.xlsx',
    'CPLT.xlsx': 'PLT - Paulista.xlsx',
}

dicionarioum = {
    'CAFI.xlsx': 'AFI - Afogados.xlsx',
}

print(dicionario.keys())
print(dicionario.values())

dict_keys(['CAFI.xlsx', 'CBAR.xlsx', 'CBJA.xlsx', 'CGAR.xlsx', 'CPES.xlsx', 'CPLT.xlsx'])
dict_values(['AFI - Afogados.xlsx', 'BAR - Barreiros.xlsx', 'BEJ - Belo Jardim.xlsx', 'GAR - Garanhuns.xlsx', 'PES - Pesqueira.xlsx', 'PLT - Paulista.xlsx'])


In [2]:
import pandas as pd
from datetime import datetime, timezone

def montar_consolidacao(dic):

    for key, value in  dic.items():
        
        campus = pd.read_excel(f'campi/{key}')

        #converte pra date utilizando o timezone utc=False não roda
        campus['data_entrada'] = pd.to_datetime(campus['data_entrada'], utc=True)
        # remove a timezone deixando so a data mas vira type obect novamente
        campus['data_entrada'] = campus['data_entrada'].dt.date
        # converte em date novamente agora sem o timezone
        campus['data_entrada'] = pd.to_datetime(campus['data_entrada'])

        campus = campus[campus['data_entrada'] < '2021-05-01']
        # campus['valor'] = pd.to_numeric(campus['valor'])

        # display(campus.info())
        campus = campus[['numero', 'categoria_codigo', 'descricao', 'data_entrada']]
#         display(campus)

        priori = pd.read_excel(f'priori/{value}')
        # display(priori)
        priori = priori[['N° Ativo', 'Nome do custo', 'Descrição']]
        priori = priori.rename(columns={'N° Ativo': 'numero'})
#         display(priori)
        
        compara = pd.merge(campus, priori, on='numero', how='outer', indicator=True)
        compara.to_excel(f'consolidacao/cosolidacao_{key}', index=False)

In [10]:
def contar_planilhas(dic):

    df_quant = pd.DataFrame()
    nome_campi = []
    lista_suap = []
    lista_priori = []

    for key, value in  dic.items():
        
        
        campus = pd.read_excel(f'campi/{key}')
        
        conta_campus = len(campus)
        lista_suap.append(conta_campus)
        nome_campi.append(key)
        
        priori = pd.read_excel(f'priori/{value}')
        conta_priori = len(priori)
        lista_priori.append(conta_priori)

    df_quant['Nome'] = nome_campi
    df_quant['Nome'] = df_quant['Nome'].str[0:4]
    df_quant['Suap'] = lista_suap
    df_quant['Priori'] = lista_priori
#     display(df_quant)
    df_quant.to_excel('quantidades.xlsx', index=False)

In [11]:
# montar_consolidacao(dicionarioum)
contar_planilhas(dicionario)